#### pip install yt-dlp openai-whisper ffmpeg-python

In [1]:
links = ['zC_kiEcthOM', 'pIcQFZzC3Kc', 'BQCrsY9PPm4', 'E6KgxZlvxbA', 'WQj3DXvVP10', 'KOwKY1fbZsE', 'p2zgRGhDmUg', 'Ok5c5-5bZek', 'Ovf1h5BmlO0', 'TdqRuIoSV8k','cJGniVYAub0','-rB4bYSFI7w', 'TrVAif9e4tc',
        'mg4Rn5AuGdw', 'F39T6O-NA6s', 'qHZvJJck4es', 'S_t1an0CAy0', '9pqLqxtlMaA', 'UuSn0zrJYho', 'YA1m44TZs2Y', 'dTX60T4n2OM'] # queraltoyo -vids del 14 al 33 + 1 boda real

In [2]:
import yt_dlp
import whisper
import numpy as np
import pandas as pd

In [3]:

# Obtención de audios de videos desde youtube
def audios(id_video):
    url = "https://www.youtube.com/watch?v=" + id_video
    fileaudio = "video_" + id_video
    
    ydl_opts = {
        'quiet': True,
        'no_warnings': True,
        "format": "bestaudio/best",
        "outtmpl": "audios/" +fileaudio + ".%(ext)s",
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    file = "audios/" + fileaudio + ".mp3"

    return file

In [4]:
# Whisper process
from datetime import datetime

def transcribeAvello(id_video):
    init = datetime.now()
    file = "audios/video_" + id_video + ".mp3"
    
    model = whisper.load_model("medium")  # tiny / base / small / medium
    result = model.transcribe(
        file,
        word_timestamps=True,
        fp16=False,
        language="es",
        no_speech_threshold=0.6,
        logprob_threshold=-1.0)
    
    subtitulos = []
    for segment in result["segments"]:
        subtitulos.append([id_video, f"{segment['start']:.2f}", f"{segment['end']:.2f}", segment['text']])
    print(file, "transcrito en ", (datetime.now() - init) / 60, " min.")
    return subtitulos

In [5]:
def descargar_audios():
    for link in links:
        print(audios(link))

In [6]:
columns = ['id_video', 'start', 'end', 'text']

In [7]:
def iniciar():
    descargar_audios()
    return transcribeAvello(links[0])

In [8]:

subs = iniciar()
data = pd.DataFrame(subs, columns = columns);
data.head(5)

audios/video_TrVAif9e4tc.mp3 transcrito en  0:00:12.922440  min.


,id_video,start,end,text
0,TrVAif9e4tc,0.00,1.36,aún telescope soy una niña.
1,TrVAif9e4tc,4.72,5.36,Una persona.
2,TrVAif9e4tc,6.48,23.74,"Edad, tenía un trinundero,"
3,TrVAif9e4tc,30.92,32.46,no te haga el lolo.
4,TrVAif9e4tc,32.60,34.06,No te haga el lolo.


In [9]:
for link in links:
    subs = transcribeAvello(link)
    new_rows = pd.DataFrame(subs, columns=columns)
    data = pd.concat([data, new_rows], ignore_index=True)

audios/video_mg4Rn5AuGdw.mp3 transcrito en  0:00:10.485352  min.
audios/video_F39T6O-NA6s.mp3 transcrito en  0:00:12.842929  min.
audios/video_qHZvJJck4es.mp3 transcrito en  0:00:10.304905  min.
audios/video_S_t1an0CAy0.mp3 transcrito en  0:00:11.404588  min.
audios/video_9pqLqxtlMaA.mp3 transcrito en  0:00:15.045721  min.
audios/video_UuSn0zrJYho.mp3 transcrito en  0:00:12.845904  min.
audios/video_YA1m44TZs2Y.mp3 transcrito en  0:00:10.935638  min.
audios/video_dTX60T4n2OM.mp3 transcrito en  0:00:09.750537  min.


In [10]:
data.to_csv("data.csv")